## Image Descriptions with Gemini 

Generate detailed textual descriptions for extracted images using Gemini 2.5 Flash.

**Prerequisites:**
-  run the Docling Data extraction notebook to extract dir like markdown, images and tables
- Google_Gemini API key set in .env file

**Output:**
- Markdown descriptions saved to `data/rag-data/images_desc/{company}/{document}/page_X.md`

### Setup and Imports

In [1]:
from dotenv import load_dotenv
load_dotenv()

from pathlib import Path
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

from PIL import Image

import base64
import io

### Configuration

In [2]:
# Paths
IMAGES_DIR = "data/rag-data/images"
OUTPUT_DESC_DIR = "data/rag-data/images_desc"

# Model configuration
MODEL_NAME = "gemini-2.5-flash"

model = ChatGoogleGenerativeAI(model=MODEL_NAME)

### Description Generation Function

In [3]:
describe_image_prompt = """Analyze this financial document page and extract meaningful data in a concise format.

For charts and graphs:
- Identify the metric being measured
- List key data points and values
- Note significant trends (growth, decline, stability)

For tables:
- Extract column headers and key rows
- Note important values and totals

For text:
- Summarize key facts and numbers only
- Skip formatting, headers, and navigation elements

Be direct and factual. Focus on numbers, trends, and insights that would be useful for retrieval."""

In [14]:
from langchain.messages import SystemMessage


def generate_image_description(image_path: Path):
    image = Image.open(image_path)
    buffered = io.BytesIO()
    image.save(buffered, format='PNG')

    image_base64 = base64.b64encode(buffered.getvalue()).decode()

    message = HumanMessage(
        content=[
            {'type': 'text', 'text': describe_image_prompt},
            {'type': 'image_url', 'image_url': f"data:image/png;base64,{image_base64}"}
        ]
    )
    system_prompt = SystemMessage('You are an AI Assistant')

    response = model.invoke([system_prompt, message])

    return response.text

In [15]:
image_path = Path(r'data\rag-data\images\meta\meta 10-k 2024\page_64.png')

response = generate_image_description(image_path)

In [22]:
# print(response)

def generate_and_save_description(image_path: Path):
    company_name = image_path.parent.parent.name
    doc_name = image_path.parent.name

    output_dir = Path(OUTPUT_DESC_DIR)/company_name/doc_name
    output_dir.mkdir(parents=True, exist_ok=True)

    desc_file = output_dir / f"{image_path.stem}.md"

    if desc_file.exists():
        return False
    
    description = generate_image_description(image_path)
    desc_file.write_text(description, encoding='utf-8')
    
    return True

In [24]:
image_path = Path(r'data\rag-data\images\meta\meta 10-k 2024\page_64.png')

response = generate_and_save_description(image_path)

In [29]:
from tqdm import tqdm

images_path = Path(IMAGES_DIR)
image_files = list(images_path.rglob("page_*.png"))

for image_path in tqdm(image_files):
    response = generate_and_save_description(image_path)


100%|██████████| 77/77 [15:34<00:00, 12.13s/it]
